***
# *Manejo de muestras de datos*
## Repaso de conceptos – Práctica 2
## Geoestadı́stica -Maestrı́a en Geomática UNLP

En un estudio global en todo el paı́s se registraron una serie de medi-
ciones meteorológicas mensuales durante el perı́odo 1981-2010. Con el
fin de caracterizar tendencias climatológicas se generó un base de datos.
En principio, como primera medida, se necesita identificar las principales
caracterı́sticas estadı́sticas de este set de datos con el fin de tener un re-
sultado rápido pero consistente. De esta manera se decidió que lo más
conveniente serı́a identificar en una tabla, por variable meteorológica, lo
siguiente:
    
***

In [64]:
################################################################
#
#   - Importacion de librerias
#   - Asignaciòn de alias
#
####################################################################

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import seaborn as sns
#import geopandas as gpd
# Preprocesado y modelado
# ==============================================================================
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
#https://towardsdatascience.com/detecting-and-treating-outliers-in-python-part-3-dcb54abaf7b0

In [85]:
# Carga de datos
    
path = "datasets/"    
archivo = "SMN_estaciones_medias_mensuales_1989_2010.xls"
df = pd.read_excel(archivo)
df.replace("S/D", np.NaN, inplace = True) #reemplazar los "S/D"

#df['Temperatura_minima_(C)'].mean()

In [86]:
#Finalmente vamos a utilizar ésta forma para cambiar el tipo de campo
rango= np.r_[8:16]
df[df.columns[rango]] = df[df.columns[rango]].astype(float)
df.dtypes
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 16 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0.1                           840 non-null    int64  
 1   Unnamed: 0                             840 non-null    int64  
 2   Estacion                               840 non-null    object 
 3   mes                                    840 non-null    object 
 4   lat                                    840 non-null    float64
 5   long                                   840 non-null    float64
 6   Altura_(m)                             840 non-null    int64  
 7   Numero                                 840 non-null    int64  
 8   Temperatura_(C)                        838 non-null    float64
 9   Temperatura_maxima_(C)                 840 non-null    float64
 10  Temperatura_minima_(C)                 818 non-null    float64
 11  Humeda

In [68]:
# Ver si reemplazamos outliers por na o por otra medida
def f(row):
    if (row[colname] < lower_lim) | (row[colname] > upper_lim):
        val = np.NaN
    else:
        val = row[colname]

    return val


In [87]:
# Ver si reemplazamos outliers por percentil

def f(row):
    if row[colname] < lower_lim:
        val = lower_lim
    elif row[colname] > upper_lim:
        val = upper_lim
    else:
        val = row[colname]

    return val

In [88]:
outliers_df = pd.DataFrame()
#type(estadisticos)
rango= np.r_[8:15]

for x in rango:
    
    colname = df.columns[x]
   # print(colname)
    Q1 = df[colname].quantile(0.25)
    Q3 = df[colname].quantile(0.75)
    IQR = Q3 - Q1
    lower_lim = Q1 - 1.5 * IQR
    upper_lim = Q3 + 1.5 * IQR
    df['outlier'] = df.apply(f, axis=1)
    df[colname] = df['outlier']

In [79]:
df['Temperatura_minima_(C)'].mean()

10.265892420537897

In [80]:
#Media

print('----------------------')
print('Media de cada variable')
print('----------------------')
df.mean(axis=0, numeric_only = True)



----------------------
Media de cada variable
----------------------


Unnamed: 0.1                               419.500000
Unnamed: 0                                 425.157143
lat                                        -31.455232
long                                       -57.741740
Altura_(m)                                 339.842857
Numero                                   87500.842857
Temperatura_(C)                             15.916527
Temperatura_maxima_(C)                      22.513214
Temperatura_minima_(C)                      10.265892
Humedad_relativa_(%)                        68.123522
Velocidad_del_Viento_(km/h)                 12.330862
Nubosidad_total_(octavos)                    3.772881
Precipitacion_(mm)                          69.334201
Frecuencia_dias_Precipitacion_>0.1_mm        6.756877
outlier                                     69.334201
dtype: float64

In [81]:
print('-------------------------')
print('Varianza de cada variable')
print('-------------------------')
df.var(axis=0) # Entrenamiento modelo PCA con escalado de los datos


-------------------------
Varianza de cada variable
-------------------------


Unnamed: 0.1                              58870.000000
Unnamed: 0                                61420.690414
lat                                          71.048752
long                                        123.912890
Altura_(m)                               241663.169556
Numero                                   127838.120688
Temperatura_(C)                              46.899090
Temperatura_maxima_(C)                       50.681690
Temperatura_minima_(C)                       44.106828
Humedad_relativa_(%)                        112.403706
Velocidad_del_Viento_(km/h)                  25.668831
Nubosidad_total_(octavos)                     0.369615
Precipitacion_(mm)                         2713.503122
Frecuencia_dias_Precipitacion_>0.1_mm         7.635285
outlier                                    2713.503122
dtype: float64

In [ ]:
#Reemplazar NaN por estadistico

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 17 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0.1                           840 non-null    int64  
 1   Unnamed: 0                             840 non-null    int64  
 2   Estacion                               840 non-null    object 
 3   mes                                    840 non-null    object 
 4   lat                                    840 non-null    float64
 5   long                                   840 non-null    float64
 6   Altura_(m)                             840 non-null    int64  
 7   Numero                                 840 non-null    int64  
 8   Temperatura_(C)                        840 non-null    float64
 9   Temperatura_maxima_(C)                 840 non-null    float64
 10  Temperatura_minima_(C)                 840 non-null    float64
 11  Humeda

In [91]:

#type(estadisticos)
rango= np.r_[8:15]

for x in rango:
    
    colname = df.columns[x]
   # print(colname)
    mean1 = df[colname].mean()
    df[colname] = df[colname].replace(np.nan, mean1)


In [93]:

df1 = df.iloc[:, np.r_[8:16]]
# Entrenamiento modelo PCA con escalado de los datos
# ==============================================================================
pca_pipe = make_pipeline(StandardScaler(), PCA())
pca_pipe.fit(df1)

# Se extrae el modelo entrenado del pipeline
modelo_pca = pca_pipe.named_steps['pca']



ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
df.columns

In [ ]:
#ejemplo: df.Per  #  df.Per.values

plt.boxplot(df.Per)
#plt.hist()
#plt.scatter()
#plt.bar()


#